# Rasa Tutorial: The Basics

### Install Dependencies

In [ ]:
!pip3 install git+https://github.com/mit-nlp/MITIE.git
!pip3 install rasa_nlu[mitie]
!pip3 install rasa_core==0.9.6
!wget https://github.com/mit-nlp/MITIE/releases/download/v0.4/MITIE-models-v0.2.tar.bz2
!tar jxf MITIE-models-v0.2.tar.bz2

# 1. Rasa NLU: Interpreters 
## (User) Utterances -> (User) Intents & Entities

### 1.a. Configure Interpreter Pipeline

In [ ]:
with open('interpreter.yml','w') as f:
  f.write('''
  language: "en"

  pipeline:
  - name: "nlp_mitie"
    model: "MITIE-models/english/total_word_feature_extractor.dat"
  - name: "tokenizer_mitie"
  - name: "ner_mitie"
  - name: "ner_synonyms"
  - name: "intent_entity_featurizer_regex"
  - name: "intent_featurizer_mitie"
  - name: "intent_classifier_sklearn"
  ''')

### 1.b. Prepare Interpreter Training Data

Format:

''##intent: intent_label"

"-example utterance for intent "

"-another example utterance with an [entity] (entity_label)"

In [ ]:
with open('interpreter_dataset.md','w') as f:
  f.write('''
  
  ## intent:intent_bye 
  - Bye 			
  - Goodbye
  - See you later
  - gotta go
  - See you
  - i'm off

  ## intent:intent_greet
  - Hi
  - Hey
  - Hello
  - Good morning
  - hi there
  - greetings

  ## intent:intent_thank
  - Thanks
  - Thank you
  - cheers
  - thanks a lot
  - cool thank you

  ## intent:intent_yes
  - yes
  - yeah
  - sure
  - absolutely
  - yep

  ## intent:intent_no
  - no
  - no way jose
  - nope
  - absolutely not
  - i don't think so

  ## intent:intent_name
  - My name is [Alice](name)  
  - I'm [Lucy](name)
  - I am [Richard](name)
  - Call me [Sally](name)
  
  ## intent:intent_question
  - I have a question about [robots](topic)
  - Can i ask you something
  - What's [AI](topic) all about anyway
  - Can you help me understand what an [LSTM](topic) is?
  - I need to ask someone about something
  - can you help me understand [the difference between a CNN and an RNN](topic)?
 
  ''')

### 1.c. Train the Interpreter

In [ ]:
!python3 -m rasa_nlu.train -c interpreter.yml -d interpreter_dataset.md -o models --fixed_model_name interpreter --verbose

2019-05-07 17:04:22 INFO     rasa_nlu.components  - Added 'nlp_mitie' to component cache. Key 'nlp_mitie-/content/MITIE-models/english/total_word_feature_extractor.dat'.
2019-05-07 17:04:22 INFO     rasa_nlu.training_data.loading  - Training data format of interpreter_dataset.md is md
2019-05-07 17:04:22 INFO     rasa_nlu.training_data.training_data  - Training data stats: 
	- intent examples: 37 (7 distinct intents)
	- Found intents: 'intent_thank', 'intent_question', 'intent_no', 'intent_name', 'intent_yes', 'intent_greet', 'intent_bye'
	- entity examples: 8 (2 distinct entities)
	- found entities: 'name', 'topic'

2019-05-07 17:04:22 INFO     rasa_nlu.model  - Starting to train component nlp_mitie
2019-05-07 17:04:22 INFO     rasa_nlu.model  - Finished training component.
2019-05-07 17:04:22 INFO     rasa_nlu.model  - Starting to train component tokenizer_mitie
2019-05-07 17:04:22 INFO     rasa_nlu.model  - Finished training component.
2019-05-07 17:04:22 INFO     rasa_nlu.model  - 

### 1.d. Test the Interpreter

In [ ]:
from rasa_core.interpreter import RasaNLUInterpreter
interpreter = RasaNLUInterpreter('models/default/interpreter')

In [ ]:
interpreter.parse(input())

my name is Mohammed


{'entities': [{'confidence': None,
   'end': 7,
   'entity': 'topic',
   'extractor': 'ner_mitie',
   'start': 3,
   'value': 'name'},
  {'confidence': None,
   'end': 19,
   'entity': 'name',
   'extractor': 'ner_mitie',
   'start': 11,
   'value': 'Mohammed'}],
 'intent': {'confidence': 0.19057546974538978, 'name': 'intent_name'},
 'intent_ranking': [{'confidence': 0.19057546974538978, 'name': 'intent_name'},
  {'confidence': 0.18999902425884263, 'name': 'intent_bye'},
  {'confidence': 0.16306338534782516, 'name': 'intent_question'},
  {'confidence': 0.14167062945137399, 'name': 'intent_no'},
  {'confidence': 0.13837872828572317, 'name': 'intent_greet'},
  {'confidence': 0.09133066232400461, 'name': 'intent_thank'},
  {'confidence': 0.08498210058684068, 'name': 'intent_yes'}],
 'text': 'my name is Mohammed'}

# 2. Rasa Core: Policies 
## (User) Intents -> (Bot) Utterances & Actions

### 2.a. Configure Policy Pipeline


In [ ]:
with open("policies.yml",'w') as f:
  f.write('''
    policies:
    - name: KerasPolicy
      epochs: 100
      max_history: 5
    - name: FallbackPolicy
      fallback_action_name: 'action_default_fallback'
    - name: MemoizationPolicy
      max_history: 5
    - name: FormPolicy
    - name: MappingPolicy
''')

### 2.b. Define the Bot's Utterances

In [ ]:
with open('domain.yml','w') as f:
  f.write('''
  
  intents:
  - intent_greet
  - intent_bye
  - intent_thank
  - intent_yes
  - intent_no
  - intent_name
  - intent_question

  entities:
  - name
  - topic

  slots:
    name:
      type: text

  actions:
  - utter_name
  - utter_thank
  - utter_greet
  - utter_salaam
  - utter_bye
  - utter_sorry
  - utter_confusion
  - utter_expertNetwork

  templates:    
    utter_name:
    - text: "What's your name?"
    - text: "My name is Ash. What can I call you?"

    utter_greet:
    - text: "Nice to you meet you {name}. How can I help?"
    - text: "Lovely to meet you {name}"
    - text: "Pleased to meet you {name}"
    - text: "Hi {name}. A pleasure meeting you"
    
    utter_salaam:
    - text: "Al-salamu 'alaykum {name}"
    - text: "Salaam {name}"
    - text: "Peace be upon you {name}"
    
    utter_bye:
    - text: "Talk to you later!"
    - text: "Bye"
    - text: "Good bye"
    - text: "Come back soon"

    utter_thank:
    - text: "My pleasure."
    - text: "Most welcome"
    - text: "Don't mention it"
    
    utter_sorry:
    - text: "I'm sorry"
    - text: "Sorry"
    - text: "forgive me"
    
    utter_confusion:
    - text: "I don't know"
    - text: "I'm not sure"
    - text: "me no understand"
    
    utter_expertNetwork:
    - text: "Interesting!  Lets see if someone in our network knows about {topic} ..."
      image: "https://s3.eu-west-2.amazonaws.com/cognitionx-assets/images/Cx_logo_Monogram_square.png"
    - text: "Let me connect you with our network of experts ..."
      image: "https://s3.eu-west-2.amazonaws.com/cognitionx-assets/images/Cx_logo_Monogram_square.png"
    - text: "sure, I'll connect you to an expert now ..."
      image: "https://s3.eu-west-2.amazonaws.com/cognitionx-assets/images/Cx_logo_Monogram_square.png"
    
  ''')

### 2.c Prepare Policy Training Data

Format:

"## story_name"

"* intent {"entity_label":"entity_value"}"

"   -   uterrance_or_action"

"   -   utterance_or_action"

In [ ]:
with open('policy_dataset.md','w') as f:
  f.write('''
  
  ## story_greet 
  * intent_greet 
   - utter_greet
   - utter_name 
   
  ## story_bye
  * intent_bye
   - utter_bye
  
  ## story_no
  * intent_no
   - utter_sorry

  ## story_thank
  * intent_thank OR intent_yes
   - utter_thank

  ## story_AIquestion
  * intent_question{"topic":"AI"}
   - utter_expertNetwork

  ## story_otherquestion
  * intent_question
   - utter_sorry
   - utter_confusion

  ## story_name
  * intent_name
   - utter_greet
   
  ## story_name
  * intent_name{"name":"Mohammed"}
   - utter_salaam
   
  ''')

### 2.d. Train the Policy

In [ ]:
!python3 -m rasa_core.train -d domain.yml -s policy_dataset.md -o models/dialogue

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/ruamel/yaml/constructor.py:283: DuplicateKeyFutureWarning: while constructing a mapping
  in "domain.yml", line 31, column 5
found duplicate key "utter_greet" with value "[]" (original value: "[]")
  in "domain.yml", line 40, column 5

To suppress this check see:
    http://yaml.readthedocs.io/en/latest/api.html#duplicate-keys

Duplicate keys will become an error in future releases, and are errors
by default when using the new API.

  warnings.warn(DuplicateKeyFutureWarning(*args))
Processed Story Blocks: 100% 10/10 [00:00<00:00, 4632.54it/s, # trackers=1]
Processed Story Blocks: 100% 10/10 [00:00<00:00, 866.79it/s, # trackers=9]
Processed Story Blocks: 100% 10/10 [00:00<00:00, 203.05it/s, # trackers=43]
Processed Story Blocks: 100% 10/10 [00:00<00:00, 69.61it/s, # trackers=31]
Processed trackers: 100% 692/692 [00:00<00:00, 860.96it/s, # actions=329]
Processed actions: 329it [00:00, 7869.96it/s, # examples=329]
Processed 

### 2.e Test the Policy

In [ ]:
from rasa_core.agent import Agent
policy = Agent.load('models/dialogue', interpreter=interpreter)

In [ ]:
policy.handle_message(input())

my name is Mohammed


[{'recipient_id': 'default', 'text': "Al-salamu 'alaykum Mohammed"}]

# Let's chat with our Rasa Bot

In [ ]:
DEFAULT_URLS = ["https://coubsecure-s.akamaihd.net/get/b54/p/coub/simple/cw_timeline_pic/7b2b17d8f24/15c695d0259a8b2231793/big_1518477107_image.jpg",
                "https://i0.wp.com/i.ytimg.com/vi/iqhKSgOKUlM/maxresdefault.jpg",
                "https://i.imgur.com/yX0HpCU.gif?noredirect"]
TEMPLATE_HTML = '''
<div class="chat-window"
<p>{}<p>
<img src = {} width="75" height="75"></img>
</div>
'''

import IPython,random

def chat():
  while True: 
    user_utterance = input()
    responses = policy.handle_message(user_utterance)
    responses_html = ''.join([TEMPLATE_HTML.format('',response["text"].split("Image:")[-1]) if "Image:" in response["text"] else TEMPLATE_HTML.format(response["text"],random.choice(DEFAULT_URLS)) for response in responses])
    IPython.display.display(IPython.display.HTML(responses_html))  
    if user_utterance == "bye": 
      break

In [ ]:
chat()

hi bot


what is your name


lol what is AI


hello


my name is Mohammed


...

yes please


ok sure


go away


good day


bye
